In [1]:
from encoder import Encoder
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from nltk.tokenize import sent_tokenize as ST

In [2]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [3]:
f = open('/home/jingjing/Desktop/big.txt', 'r')
data = f.read()
splat = data.split("\n\n")

In [4]:
paragraphs = splat

In [5]:
sentences = ['The Moon is filled wit craters.', 'It has no light of its own.', 'It gets its light from the Sun.']
data = []
for p in paragraphs:
    nl = ST(p)
    if len(nl) > 1:
        data.append(ST(p))


In [6]:
f = Encoder()

f.zero_grad()

f.set_w2v_path(W2V_PATH)


In [7]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(dim, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [8]:
def loss_fn(pred, target):
    mask = 1 - torch.diag(torch.ones(pred.size(1)))
    npred = pred * mask
    s_pred = F.softmax(pred, -1)
    losses = F.binary_cross_entropy_with_logits(s_pred, target, size_average=False)
    return losses

In [9]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    embeddings = f.encode(sentences, 400, True, False)
    targets = make_target(1, len(sentences))
    loss = loss_fn(embeddings, targets.float())
    print("loss before training: ", loss)

Found 20(/20) words with w2v vectors
Vocab size : 20
loss before training:  tensor(7.0856)


In [10]:
optimizer = optim.Adam(f.parameters(), lr=0.0005)
nn.utils.clip_grad_norm_(f.parameters(), 5)

0.0

In [11]:
for epoch in range(2):
    for instance in data:
        optimizer.zero_grad()
        if epoch == 0:
            f.build_vocab(instance, True)
        targets = make_target(1, len(instance))
        scores = f.encode(instance, len(instance))
        loss = loss_fn(scores, targets.float())
        print(loss)
        loss.backward()
        optimizer.step()

Found 46(/46) words with w2v vectors
Vocab size : 46
tensor(7.5229)
Found 44(/44) words with w2v vectors
Vocab size : 44
tensor(3.6244)
Found 40(/40) words with w2v vectors
Vocab size : 40
tensor(12.2651)
Found 55(/55) words with w2v vectors
Vocab size : 55
tensor(3.5846)
Found 85(/85) words with w2v vectors
Vocab size : 85
tensor(36.8524)
Found 75(/75) words with w2v vectors
Vocab size : 75
tensor(12.5819)
Found 45(/45) words with w2v vectors
Vocab size : 45
tensor(3.1812)
Found 45(/45) words with w2v vectors
Vocab size : 45
tensor(3.0520)
Found 35(/35) words with w2v vectors
Vocab size : 35
tensor(3.2903)
Found 45(/45) words with w2v vectors
Vocab size : 45
tensor(6.2390)
Found 86(/86) words with w2v vectors
Vocab size : 86
tensor(12.0957)
Found 44(/44) words with w2v vectors
Vocab size : 44
tensor(3.0130)
Found 62(/62) words with w2v vectors
Vocab size : 62
tensor(13.0608)
Found 65(/65) words with w2v vectors
Vocab size : 65
tensor(11.8934)
Found 43(/43) words with w2v vectors
Vocab

KeyboardInterrupt: 

In [12]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    embeddings = f.encode(sentences, len(sentences))
    targets = make_target(1, len(sentences))
    loss = loss_fn(embeddings, targets.float())
    print("loss after training: ", loss)

Found 20(/20) words with w2v vectors
Vocab size : 20
loss after training:  tensor(6.2848)
